In [1]:
%matplotlib inline
import sys
from refer import REFER
import cv2
import skimage.io as io
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import json

from textblob import TextBlob
from textblob_aptagger import PerceptronTagger

In [2]:
def show_image(image):
    height = image.shape[0]
    width = image.shape[1]
    
    margin=50 # pixels
    dpi=100. # dots per inch

    figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
    left = margin/dpi/figsize[0] #axes ratio
    bottom = margin/dpi/figsize[1]

    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)

    plt.imshow(image)
    plt.show()

In [3]:
data_root = 'data'  # contains refclef, refcoco, refcoco+, refcocog and images
dataset = 'refcoco'
splitBy = 'unc'
refer = REFER(data_root, dataset, splitBy)

loading dataset refcoco into memory...
creating index...
index created.
DONE (t=12.52s)


In [16]:
ref_id=254
ref = refer.Refs[ref_id]
image_file_name = '../refer/data/images/mscoco/images/train2014' + '/' + refer.Imgs[ref['image_id']]['file_name']
# x,y,w,h=refer.getRefBox(ref_id)
# image = cv2.imread(image_file_name)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# show_image(image)

In [4]:
filter_keywords = set([
                   'left', 'right', 'top', 'bottom', 
                   'middle', 'big', 'small', 'long', 'short', 'tall', 'huge', 'tiny',
                   'blue', 'green', 'red', 'yellow', 'pink', 'purple', 'orange', 'black', 'gray', 'white', 'brown'])

filtered_ref_ids = set()
filtered_ref_ids_sentences = dict()
train_ref_ids = refer.getRefIds()
for ref_id in tqdm(train_ref_ids):
    ref = refer.Refs[ref_id]
    for sentence in ref['sentences']:
        if len(set(sentence['tokens']) & filter_keywords) > 0:
            filtered_ref_ids.add(ref_id)
            filtered_ref_ids_sentences[ref_id] = sentence
len(filtered_ref_ids)

100%|██████████| 50000/50000 [00:00<00:00, 195467.20it/s]


43025

In [5]:
filtered_ref_ids_sentences

{0: {'raw': 'blue shirt',
  'sent': 'blue shirt',
  'sent_id': 2,
  'tokens': ['blue', 'shirt']},
 1: {'raw': 'woman greyshirt right',
  'sent': 'woman greyshirt right',
  'sent_id': 5,
  'tokens': ['woman', 'greyshirt', 'right']},
 2: {'raw': 'middle standing up',
  'sent': 'middle standing up',
  'sent_id': 8,
  'tokens': ['middle', 'standing', 'up']},
 3: {'raw': 'woman sitting on right',
  'sent': 'woman sitting on right',
  'sent_id': 11,
  'tokens': ['woman', 'sitting', 'on', 'right']},
 4: {'raw': 'left person',
  'sent': 'left person',
  'sent_id': 13,
  'tokens': ['left', 'person']},
 6: {'raw': 'left urinal',
  'sent': 'left urinal',
  'sent_id': 20,
  'tokens': ['left', 'urinal']},
 7: {'raw': 'right urinal',
  'sent': 'right urinal',
  'sent_id': 23,
  'tokens': ['right', 'urinal']},
 8: {'raw': 'right most vase',
  'sent': 'right most vase',
  'sent_id': 25,
  'tokens': ['right', 'most', 'vase']},
 9: {'raw': 'middle', 'sent': 'middle', 'sent_id': 27, 'tokens': ['middle']}

In [ ]:
second_filtered_ref_ids = set()
subjects = dict()
boxes = dict()
image_ids = dict()
image_file_names = dict()
sentences = dict()
ref_tags = dict()

for ref_id in tqdm(list(filtered_ref_ids)):
    ref = refer.Refs[ref_id]
    x,y,w,h=refer.getRefBox(ref_id)
    sentence = filtered_ref_ids_sentences[ref_id]['sent']
    blob = TextBlob(sentence, pos_tagger=PerceptronTagger())
    tags = blob.tags
    subject = None
    for token, tag in tags:
        if token not in filter_keywords and tag == 'NN':
            subject = token
            break
    if subject is not None:
        second_filtered_ref_ids.add(ref_id)
        subjects[ref_id] = subject
        boxes[ref_id] = (x,y,w,h)
        image_ids[ref_id] = ref['image_id']
        image_file_name = '../refer/data/images/mscoco/images/train2014' + '/' + refer.Imgs[ref['image_id']]['file_name']
        image_file_names[ref_id] = image_file_name
        sentences[ref_id] = sentence
        ref_tags[ref_id] = tags
        
my_refer_data = {'ref_ids': list(second_filtered_ref_ids), 
                 'subjects': subjects, 
                 'boxes': boxes, 
                 'image_ids': image_ids,
                 'image_file_names': image_file_names,
                 'sentences': sentences,
                 'ref_tags': ref_tags}

with open('my_refer_data.pickle', 'wb') as file:
    pickle.dump(my_refer_data, file)

100%|██████████| 43025/43025 [1:49:35<00:00,  6.54it/s]


In [ ]:
with open('my_refer_data.pickle', 'rb') as file:
    my_refer_data = pickle.load(file)

In [12]:
x=3

In [16]:
x

3

In [17]:
len(my_refer_data['ref_ids'])

40062